In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from edward.models import Bernoulli, Normal

In [112]:
ed.set_seed(42)

RuntimeError: Seeding is not supported after initializing part of the graph. Please move set_seed to the beginning of your code.

In [114]:
%matplotlib inline

In [3]:
def build_toy_dataset(N, noise_std=0.1):
  D = 1
  X = np.linspace(-6, 6, num=N)
  y = np.tanh(X) + np.random.normal(0, noise_std, size=N)
  y[y < 0.5] = 0
  y[y >= 0.5] = 1
  X = (X - 4.0) / 4.0
  X = X.reshape((N, D))
  return X, y

In [81]:
N = 40 # num data points
D = 2 # num features

In [82]:
# DATA
X_train, y_train = build_toy_dataset(N)

In [83]:
# MODEL
X = tf.placeholder(tf.float32, [N, D])
w = Normal(loc=tf.zeros(D), scale=3.0 * tf.ones(D))
b = Normal(loc=tf.zeros([]), scale=3.0 * tf.ones([]))
y = Bernoulli(logits=ed.dot(X, w) + b)

In [84]:
print(ed.get_session().run(w))
print(ed.get_session().run(b))
#ed.get_session().run(y)

[-3.61462212  0.63671732]
1.2365


In [85]:
# INFERENCE
qw_loc = tf.Variable(tf.random_normal([D]))
qw_scale = tf.nn.softplus(tf.Variable(tf.random_normal([D])))
qb_loc = tf.Variable(tf.random_normal([]) + 10)
qb_scale = tf.nn.softplus(tf.Variable(tf.random_normal([])))

In [88]:
qw = Normal(loc=qw_loc, scale=qw_scale)
qb = Normal(loc=qb_loc, scale=qb_scale)

In [89]:
inference = ed.KLqp({w: qw, b: qb}, data={X: X_train, y: y_train})
inference.initialize(n_print=10, n_iter=600)

In [90]:
tf.global_variables_initializer().run()

In [91]:
# Set up figure.
fig = plt.figure(figsize=(8, 8), facecolor='white')
ax = fig.add_subplot(111, frameon=False)
plt.ion()
plt.show(block=False)

In [98]:
inputs = np.linspace(-5, 3, num=800, dtype=np.float32).reshape((400, 2))

In [100]:
inputs.shape

(400, 2)

In [104]:
ed.get_session().run(qw)

array([ 0.08244229, -0.40549904], dtype=float32)

In [96]:
# Build samples from inferred posterior.
n_samples = 50
inputs = np.linspace(-5, 3, num=400, dtype=np.float32).reshape((400, 2))
probs = tf.stack([tf.sigmoid(ed.dot(inputs, qw.sample()) + qb.sample())
                  for _ in range(n_samples)])

for t in range(inference.n_iter):
  info_dict = inference.update()
  inference.print_progress(info_dict)

  if t % inference.n_print == 0:
    outputs = probs.eval()

    # Plot data and functions
    plt.cla()
    ax.plot(X_train[:], y_train, 'bx')
    for s in range(n_samples):
      ax.plot(inputs[:], outputs[s], alpha=0.2)

    ax.set_xlim([-5, 3])
    ax.set_ylim([-0.5, 1.5])
    plt.draw()
    plt.pause(1.0 / 60.0)

ValueError: cannot reshape array of size 400 into shape (400,2)

In [105]:
print(ed.get_session().run(qw_scale))
print(ed.get_session().run(qw_loc))
print(ed.get_session().run(qb_loc))
print(ed.get_session().run(qb_scale))

[ 0.17155297  1.03258669]
[ 0.08493871  0.74098688]
10.2736
0.239139


In [107]:
print(ed.get_session().run(qw_scale))
print(ed.get_session().run(qw_loc))
print(ed.get_session().run(qb_loc))
print(ed.get_session().run(qb_scale))

[ 0.17155297  1.03258669]
[ 0.08493871  0.74098688]
10.2736
0.239139


In [110]:
print(ed.get_session().run(qw))

[-0.20231941  1.55481994]
